# Тестирование

## Зачем писать тесты



* Проверить что код ведет себя как надо во всех случаях (даже в крайних).
* Проверить что ничего не сломалось после изменения кода/добавления новой функциональности.
* Понять что именно сломалось (в этом помогают юнит-тесты).
* **TDD (Test Driven Development)** - один из подходов к разработке: сначала пишутся тесты, потом функционал, который их проходит.



## Doctest

Модуль doctest 
* находит части докстрингов, которые выглядят как интерактивные сессии питона 
* выполняет эти сессии
* проверяет, что результат выполнения кода соответствует написанному в документации

In [0]:
import datetime
import re
from typing import Optional

In [0]:
class RuDateParser:

    # Optional значит, что возвращаеся либо объект указанного типа, либо None
    def parse_numeric(self, date: str) -> Optional[datetime.datetime]: 
        """ 
        Парсит даты в формате dd-mm-yyyy

        >>> RuDateParser().parse_numeric('01-12-2010')
        datetime.datetime(2010, 12, 1, 0, 0)

        >>> RuDateParser().parse_numeric('01/12/2010')

        >>> RuDateParser().parse_numeric('не дата')
        """
        if not isinstance(date, str):
            raise TypeError
        # \d - digit, то же самое, что [0-9]
        elif not re.match('\d{2}-\d{2}-\d{4}', date): 
            return None
        date_splitted = date.split('-')
        day = int(date_splitted[0])
        month = int(date_splitted[1])
        year = int(date_splitted[2]) 
        return datetime.datetime(day=day, month=month, year=year)


In [0]:
parser = RuDateParser()
parser.parse_numeric('01-12-2010')

datetime.datetime(2010, 12, 1, 0, 0)

In [0]:
import doctest

In [0]:
if __name__ == '__main__':
    doctest.testmod()

## Юнит тесты

* каждый самосоятельный блок кода (класс, метод, функция) тестируется отдельно и изолированно

Давайте напишем юнит-тесты для метода parse_numeric с использованием всторенной библиотеки unittest:
* тесты в unittest организуются в группы внутри класса TestCase
* обычно - один Test Case для каждого класса/функции (но не всегда)
* иногда имеет смысл делать общий TestCase для всего модуля
* отдельный тест на каждый вариант развития событий 
* тесты оформляются в виде методов класса, названия методов начинаются со слова test
* можно писать свои вспомогательные методы (которые не являются тестами), главное, чтобы их называния не начинались с test

In [0]:
import unittest
class RuDateParserTestCase(unittest.TestCase):
    def setUp(self):
        self.parser = RuDateParser()

    # тестируем поведение при правильных входных данных
    def test_parse_numeric_matching_string(self):
        self.assertEqual(datetime.datetime(day=12, month=1, year=2020), 
                         self.parser.parse_numeric('12-01-2020'))
        
    # тестируем поведение при вводе строки, не содержащей дату в нужном формате
    def test_parse_numeric_unmatching_string(self):
        self.assertEqual(None, self.parser.parse_numeric('12/01/2020'))
        
    # тестируем поведение при неправильном типе входных данных
     def test_parse_numeric_incorrect_input_type(self):
        self.assertRaises(TypeError, self.parser.parse_numeric, 123)

# запустить все тесты
if __name__ == '__main__':
    # unittest.main()  # jupyter
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # colab

...
----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


### Assert

* Стандартная инструкция assert - проверить, истинно ли какое-то выражение в питоне. 
* Если нет, то поднимается AssertionError. 

In [0]:
a = 2
assert a == 1, 'сообщение которое выведется вместе с AssertionError'

AssertionError: ignored

В тестах можно использовать встроенный assert или методы класса unittest.TestCase. Если условие под assert не выполняется, то тест помечается непройденным. Полный список в [документации](https://docs.python.org/3/library/unittest.html#unittest.TestCase). 
* ***assertEqual***_(first, second, msg=None)_  
    Test that first and second are equal. If the values do not compare equal, the test will fail.

* ***assertNotEqual***_(first, second, msg=None)_  
    Test that first and second are not equal. If the values do compare equal, the test will fail.

* ***assertTrue***_(expr, msg=None)_      
  ***assertFalse***_(expr, msg=None)_      
    Test that expr is true (or false).

* ***assertIs***_(first, second, msg=None)_   
  ***assertIsNot***_(first, second, msg=None)_    
    Test that first and second evaluate (or don’t evaluate) to the same object.

* ***assertIn***_(first, second, msg=None)_    
  ***assertNotIn***_(first, second, msg=None)_     
    Test that first is (or is not) in second.

* ***assertRaises***_(exception, callable, *args, **kwds)_   
    Test that an exception is raised when callable is called with any positional or keyword arguments that are also passed to assertRaises(). The test passes if exception is raised, is an error if another exception is raised, or fails if no exception is raised. To catch any of a group of exceptions, a tuple containing the exception classes may be passed as exception.

### setUp и tearDown

 Вспомогательные методы, которые используются, чтобы настроить среду тестирования. Нужны для того, чтобы убедиться, что каждый тест работает изолированно и на него не влияет, например, результат работы прошлого теста
 * ***setUp()*** - вызывается перед запуском каждого теста.
 * ***tearDown()*** - вызывается после запуска каждого теста. 

In [0]:
class MyTest(TestCase):

    # создать временную папку перед запуском теста
    def setUp(self):
    self.test_dir = TemporaryDirectory()

    # удалить папку после выполнения теста
    def tearDown(self):
    self.test_dir.cleanup()

    # дальше идут сами тесты, в которых что-то делается с этой папкой 

**Задание:** 
1. parse_numeric
    1. Подумать, какой ввод сломает метод parse_numeric (он выдаст ошибку, но не поднятую нами TypeError), что мы не учли при написании метода
    2. Исправить метод  
    3. Дописать соответсвующие тесты
2. parse_natural
    1. Реализовать метод parse_natural (по аналогии с parse_numeric), сделать так, чтобы он не ломался при любом вводе. 
    2. Запустить doctest и проверить, что все хорошо. 
    2. Написать к нему тесты используя unittest. 

In [0]:
class RuDateParser:

    def parse_numeric(self, date: str) -> Optional[datetime.datetime]: 
        """ 
        Парсит даты в формате dd-mm-yyyy

        >>> RuDateParser().parse_numeric('01-12-2010')
        datetime.datetime(2010, 12, 1, 0, 0)

        >>> RuDateParser().parse_numeric('01/12/2010')

        >>> RuDateParser().parse_numeric('не дата')
        """
        if not isinstance(date, str):
            raise TypeError
        # \d - digit, то же самое, что [0-9]
        elif not re.match('\d{2}-\d{2}-\d{4}', date): 
            return None
        date_splitted = date.split('-')
        day = int(date_splitted[0])
        month = int(date_splitted[1])
        year = int(date_splitted[2]) 
        return datetime.datetime(day=day, month=month, year=year)

    def parse_natural(self, date: str) -> Optional[datetime.datetime]: 
        """ 
        Парсит даты в формате dd MMMM yyyy на русском языке

        >>> RuDateParser().parse_natural('1 января 2010')
        datetime.datetime(2010, 12, 1, 0, 0)

        >>> RuDateParser().parse_natural('1 Января 2010')
        datetime.datetime(2010, 12, 1, 0, 0)

        >>> RuDateParser().parse_natural('01 января 2010')
        datetime.datetime(2010, 12, 1, 0, 0)
        
        >>> RuDateParser().parse_natural('не дата')
        """
        pass